In [1]:
import os
import cv2
from glob import glob
import numpy as np
import pandas as pd
from kitti_utils import *

### Number of Files

In [2]:
image_files = '../2011_09_26_drive_0015_sync/2011_09_26/2011_09_26_drive_0015_sync/'

# Get the left and right images.
right_image_path = sorted(glob(os.path.join(image_files,'image_02/data/*.png')))
left_image_path = sorted(glob(os.path.join(image_files,'image_03/data/*.png')))

# Get LiDAR data
bin_paths = sorted(glob(os.path.join(image_files,'velodyne_points/data/*.bin')))

#Get GPS/Imu data
oxts_path = sorted(glob(os.path.join(image_files,r'oxts/data/*.txt')))

print(f'Number of right images: ',{len(right_image_path)})
print(f'Number of left images: ', {len(left_image_path)})
print(f'Number of LiDAR' ,{len(bin_paths)})
print(f'Number of GPS data', {len(oxts_path)})

Number of right images:  {297}
Number of left images:  {297}
Number of LiDAR {297}
Number of GPS data {297}


### Loading camera calibration data

In [3]:
with open ('../2011_09_26_calib/2011_09_26/calib_cam_to_cam.txt') as f:
    calib = f.readlines()
# Projection matrices
P_rect_cam2 = np.array([float(x) for x in calib[25].strip().split(' ')[1:]]).reshape((3,4))

# get rectified rotation matrices (left camera --> rectified left camera)
R_ref0_rect2 = np.array([float(x) for x in calib[24].strip().split(' ')[1:]]).reshape((3, 3,))

# add (0,0,0) translation and convert to homogeneous coordinates
R_ref0_rect2 = np.insert(R_ref0_rect2, 3, values=[0,0,0], axis=0)
R_ref0_rect2 = np.insert(R_ref0_rect2, 3, values=[0,0,0,1], axis=1)


# get rigid transformation from Camera 0 (ref) to Camera 2
R_2 = np.array([float(x) for x in calib[21].strip().split(' ')[1:]]).reshape((3,3))
t_2 = np.array([float(x) for x in calib[22].strip().split(' ')[1:]]).reshape((3,1))

# get cam0 to cam2 rigid body transformation in homogeneous coordinates
T_ref0_ref2 = np.insert(np.hstack((R_2, t_2)), 3, values=[0,0,0,1], axis=0)

### Get the LiDAR and IMU data

In [4]:
T_velo_ref0 = get_rigid_transformation('../2011_09_26_calib/2011_09_26/calib_velo_to_cam.txt')
T_imu_velo = get_rigid_transformation('../2011_09_26_calib/2011_09_26/calib_imu_to_velo.txt')

## Load Labels/ Annotations

In [38]:
from parseTrackletXML import *
xml_file = '../2011_09_26_drive_0015_tracklets/2011_09_26/2011_09_26_drive_0015_sync/tracklet_labels.xml'
test = parseXML(xml_file)

parsing tracklet file ../2011_09_26_drive_0015_tracklets/2011_09_26/2011_09_26_drive_0015_sync/tracklet_labels.xml
file contains 36 tracklets
loaded 36 tracklets


In [43]:
a = []
for i in test:
    a.append(i.firstFrame)
print(a)

[0, 0, 10, 28, 26, 44, 53, 55, 53, 68, 101, 139, 165, 170, 170, 148, 170, 170, 186, 197, 204, 215, 209, 228, 241, 239, 254, 257, 259, 264, 256, 0, 141, 106, 63, 167]


In [37]:
def extractDimensions(filename):
    dimensions = parseXML(filename)
    h,w,l = [],[],[]
    for i in dimensions:
        h.append(i.size[0])
        w.append(i.size[1])
        l.append(i.size[2])
    return h,w,l
h,w,l = extractDimensions(xml_file)

parsing tracklet file ../2011_09_26_drive_0015_tracklets/2011_09_26/2011_09_26_drive_0015_sync/tracklet_labels.xml
file contains 36 tracklets
loaded 36 tracklets


In [49]:
'''from Mask_RCNN.mrcnn.utils import Dataset
class KITTIDataset(Dataset):
    def load_dataset(self, dataset_images,trackletFile=xml_file):
        tracklets = parseXML(trackletFile)
        for i in dataset_images:
            image_id = i[:-4]
            #left_images = sorted(os.path.join(image_files,'image_02/data/'))
            right_images = os.path.join(image_files, 'image_03/data/*.png')
            
            for track in tracklets:
                first_frame = int(track.firstFrame)
            print(i)

k = KITTIDataset()
k.load_dataset(image_files,xml_file)  
'''              
                
        




"from Mask_RCNN.mrcnn.utils import Dataset\nclass KITTIDataset(Dataset):\n    def load_dataset(self, dataset_images,trackletFile=xml_file):\n        tracklets = parseXML(trackletFile)\n        for i in dataset_images:\n            image_id = i[:-4]\n            #left_images = sorted(os.path.join(image_files,'image_02/data/'))\n            right_images = os.path.join(image_files, 'image_03/data/*.png')\n            \n            for track in tracklets:\n                first_frame = int(track.firstFrame)\n            print(i)\n\nk = KITTIDataset()\nk.load_dataset(image_files,xml_file)  \n"